In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score

import load_data
from preprocess_data import ChestXRay14

In [29]:
# Load dataset using existing functions
train_loader, ds_train, ds_test = load_data.get_data()

# Check a sample batch
for batch in train_loader:
    print(f"Batch image shape: {batch['image'].shape}, Batch label shape: {batch['label'].shape}")
    break

Exception: The API for Deep Lake 4.0 has changed significantly, including the `load` method being replaced by `open`. To continue using Deep Lake 3.x, use `pip install "deeplake<4"`. For information on migrating your code, see https://docs.deeplake.ai/latest/details/v3_conversion/

In [ ]:
class ChestXRayCNN(nn.Module):
    def __init__(self, num_classes=14):
        super(ChestXRayCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # Assuming input image is resized to 64x64
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = torch.flatten(x, start_dim=1)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation here since we'll use BCEWithLogitsLoss

        return x

In [ ]:
# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Instantiate model and move to device
num_classes = 14
model = ChestXRayCNN(num_classes=num_classes).to(device)

# Define loss function (BCEWithLogitsLoss for multilabel classification)
criterion = nn.BCEWithLogitsLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, dataloader, criterion, optimizer, device, num_epochs=50):
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

# Train the model
train(model, train_loader, criterion, optimizer, device, num_epochs=50)

In [ ]:
def evaluate(model, dataloader, device):
    model.eval()

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in dataloader:
            images = batch['image'].to(device)
            labels = batch['label'].cpu().numpy()  # Convert to numpy

            outputs = model(images)
            predictions = torch.sigmoid(outputs).cpu().numpy()

            all_labels.append(labels)
            all_preds.append(predictions)

    # Convert to numpy arrays
    all_labels = np.vstack(all_labels)
    all_preds = np.vstack(all_preds)

    # Compute AUC-ROC per class
    auc_scores = []
    for i in range(all_labels.shape[1]):
        auc = roc_auc_score(all_labels[:, i], all_preds[:, i])
        auc_scores.append(auc)

    print(f"Mean AUC-ROC: {np.mean(auc_scores):.4f}")
    for i, auc in enumerate(auc_scores):
        print(f"Class {i+1} AUC-ROC: {auc:.4f}")

# Evaluate the model
evaluate(model, train_loader, device)